In [ ]:
import numpy as np
np.random.seed(1)
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data()
amount_of_train_words = 1000
y_train = y_train.reshape((len(y_test), 1))
y_test = y_test.reshape((len(y_test), 1))
words_indexes = imdb.get_word_index()
rev_words_indexes = dict()
for word, ind in words_indexes.items():
    rev_words_indexes[ind] = word
    
amount_of_examples = len(x_train) #25000
input_feedback = np.zeros((amount_of_examples, amount_of_train_words))
for ex in range(amount_of_examples):
    for ind in x_train[ex]:
        if ind < amount_of_train_words:
            input_feedback[ex][ind] = 0.1
            

test_feedback = np.zeros((len(x_test), amount_of_train_words))
for ex in range(len(x_test)):
    for ind in x_test[ex]:
        if ind < amount_of_train_words:
            test_feedback[ex][ind] = 0.1


            
alpha, iterations, hidden_size = (10000, 300, 100)
batch_size = 500
weights_0_1 = 0.2 * np.random.random((amount_of_train_words, hidden_size)) - 0.1
weights_1_2 = 2 * np.random.random((hidden_size, 1)) - 1

relu = lambda x: (x > 0) * x
relu2deriv = lambda x: (x > 0)
sigmoid = lambda x: 1 / (1 + np.exp(-x))
for j in range(iterations):
    correct_cnt, error = 0, 0.0
    for i in range(len(input_feedback) // batch_size):
        batch_st, batch_end = (i * batch_size, (i + 1) * batch_size)
        layer_0 = input_feedback[batch_st: batch_end]
        layer_1 = relu(layer_0.dot(weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = layer_1.dot(weights_1_2)
        
#         print(y_train[batch_st: batch_end].shape, len(input_feedback) // batch_size)
        for k in range(batch_size):
#             print(layer_2[k: k + 1], y_train[batch_st + k: batch_st + k + 1])
            correct_cnt += int(abs(layer_2[k: k + 1] - y_train[batch_st + k: batch_st + k + 1]) < 0.5)
        
        
        layer_2_delta = (y_train[batch_st: batch_end] - layer_2) / (batch_size * layer_2.shape[0]) # !!!!!!!!!
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        layer_1_delta *= dropout_mask
        error += np.sum(layer_2_delta**2, axis=0)
        
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta) # !!!!!!!!!!
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta) # !!!!!!!!!!
#         print('@ ', i)
    test_correct_cnt = 0
    for i in range(len(test_feedback)):
        layer_0 = test_feedback[i: i + 1]
        layer_1 = relu(layer_0.dot(weights_0_1))
        layer_2 = layer_1.dot(weights_1_2)
        test_correct_cnt += int(abs(layer_2 - y_test[i: i + 1]) < 0.5)
    print('I:\t', j)
    print('Train-Acc:\t', correct_cnt / float(len(input_feedback)))
    print('Test-Acc:\t', test_correct_cnt / len(test_feedback))
        
        
            


I:	 0
Train-Acc:	 0.48776
Test-Acc:	 0.5
I:	 1
Train-Acc:	 0.5
Test-Acc:	 0.5
I:	 2
Train-Acc:	 0.49808
Test-Acc:	 0.5
I:	 3
Train-Acc:	 0.5
Test-Acc:	 0.5
I:	 4
Train-Acc:	 0.5
Test-Acc:	 0.5
I:	 5
Train-Acc:	 0.5
Test-Acc:	 0.5
I:	 6
Train-Acc:	 0.5
Test-Acc:	 0.5
